In [ ]:
import pandas as pd
import re
hotspot_data = pd.read_csv("hotspot_AQUA_streaming.csv")
hotspot_data = hotspot_data.sample(frac=1).reset_index(drop=True)

import pymongo
from pymongo import MongoClient
from datetime import datetime, timedelta
import random


def getNextClimateDay():
    #gets the date of the latest climate record
    client = MongoClient('192.168.0.108', 27017) #change to local ip
    db = client.fit3182_db 
    climate = db["climate"]
    hotspots = db['hotspots']
    latest_record = climate.find_one({}, sort=[('date', pymongo.DESCENDING)])
    latest_date = latest_record['date']
    client.close()
    return latest_date


latest_date = getNextClimateDay()
import pandas as pd
import random
import time
from kafka3 import KafkaProducer
import datetime as dt
from time import sleep
from json import dumps

hostip = "192.168.0.108" # change to local ip

def pushData(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                 api_version=(0,10))
        
    except Exception as ex:
        print('could not connect to Kafka')
        print(str(ex))
    finally:
        return _producer
        
        
if __name__ == '__main__':
    topic = 'Producer2'
    
    print('starting kafka stream')
    
    producer = connect_kafka_producer()
    
    #randomly order hotspot data
    hotspot_data = hotspot_data.sample(frac=1).reset_index(drop=True)
    
    
    latest_date = latest_date + timedelta(days=1)
    
    for index, row in hotspot_data.iterrows():
        df = row
        df['created_on'] = latest_date.isoformat()
        df['producer'] = topic
        record_json = df.to_json()
        pushData(producer,topic, 'parsed', record_json)
        latest_date = latest_date + timedelta(minutes=288) #simulate 4.8 hours 
        sleep(2) #simulate 4.8 hours
        
    


starting kafka stream
Message published successfully. Data: {"latitude":-36.4226,"longitude":143.0946,"confidence":56.0,"surface_temperature_celcius":39.0,"created_on":"2024-11-22T00:00:00","producer":"Producer2"}
Message published successfully. Data: {"latitude":-35.4509,"longitude":143.1522,"confidence":73.0,"surface_temperature_celcius":47.0,"created_on":"2024-11-22T04:48:00","producer":"Producer2"}
Message published successfully. Data: {"latitude":-36.3728,"longitude":141.3641,"confidence":71.0,"surface_temperature_celcius":46.0,"created_on":"2024-11-22T09:36:00","producer":"Producer2"}
Message published successfully. Data: {"latitude":-36.3278,"longitude":141.082,"confidence":54.0,"surface_temperature_celcius":39.0,"created_on":"2024-11-22T14:24:00","producer":"Producer2"}
Message published successfully. Data: {"latitude":-36.3313,"longitude":147.6089,"confidence":74.0,"surface_temperature_celcius":52.0,"created_on":"2024-11-22T19:12:00","producer":"Producer2"}
Message published s